In [1]:
!pip install ultralytics opencv-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00a 0:00:01


In [2]:
pip install lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from ultralytics import YOLO
import cv2
import csv

# =========================================
# PATHS
# =========================================
VIDEO_PATH = "/kaggle/input/video-traffic-1080/vehicles_stack_1080p.mp4"
COCO_MODEL_PATH = "yolov8s.pt"
AUTO_MODEL_PATH = "/kaggle/input/auto-model-1/pytorch/default/1/best (1).pt"

OUTPUT_VIDEO = "/kaggle/working/final_output_fixed_v1.mp4"
CSV_PATH = "/kaggle/working/violations_v1.csv"

# =========================================
# PARAMETERS (TUNABLE)
# =========================================
COCO_VEHICLES = {"car", "bus", "truck", "motorcycle"}

STOP_LINE_Y_RATIO = 0.60
STOPPED_DIST_PX = 3

FRONT_N = 4                     # ✔ only front vehicles
RED_RATIO_THRESHOLD = 0.6       # ✔ % stopped → RED
GREEN_HOLD_FRAMES = 15          # ✔ green buffer

QUEUE_DEPTH_RATIO = 0.35
IOU_MATCH = 0.5
CONF_MARGIN = 0.15

# =========================================
# LOAD MODELS
# =========================================
coco_model = YOLO(COCO_MODEL_PATH)
auto_model = YOLO(AUTO_MODEL_PATH)

# =========================================
# IOU FUNCTION
# =========================================
def iou(a, b):
    xA = max(a[0], b[0])
    yA = max(a[1], b[1])
    xB = min(a[2], b[2])
    yB = min(a[3], b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (a[2]-a[0])*(a[3]-a[1])
    areaB = (b[2]-b[0])*(b[3]-b[1])
    return inter / (areaA + areaB - inter + 1e-6)

# =========================================
# VIDEO SETUP
# =========================================
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

STOP_LINE_Y = int(H * STOP_LINE_Y_RATIO)
QUEUE_DEPTH_PX = int(H * QUEUE_DEPTH_RATIO)

out = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W, H)
)

# =========================================
# STATE
# =========================================
last_front_y = {}
was_stopped = {}
violators = set()

signal_state = "RED"
green_hold = 0

violation_count = 0
violation_log = []
frame_idx = 0

# =========================================
# MAIN LOOP
# =========================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    time_sec = round(frame_idx / fps, 2)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # -------- COCO DETECTION + TRACKING --------
    coco_res = coco_model.track(
        frame_rgb,
        persist=True,
        conf=0.4,
        device=0,
        verbose=False
    )[0]

    vehicle_boxes = []

    if coco_res.boxes is not None:
        for b in coco_res.boxes:
            label = coco_res.names[int(b.cls)]
            if label in COCO_VEHICLES and b.id is not None:
                x1, y1, x2, y2 = map(int, b.xyxy[0])
                vehicle_boxes.append([
                    x1, y1, x2, y2,
                    label,
                    float(b.conf),
                    int(b.id)
                ])

    # -------- AUTO RICKSHAW REFINEMENT --------
    crops, offsets = [], []
    for x1, y1, x2, y2, label, _, _ in vehicle_boxes:
        if label in {"car", "truck"}:
            crop = frame_rgb[y1:y2, x1:x2]
            if crop.size > 0:
                crops.append(crop)
                offsets.append((x1, y1))

    auto_res = auto_model.predict(
        crops,
        conf=0.25,
        device=0,
        verbose=False
    ) if crops else []

    for i, res in enumerate(auto_res):
        if res.boxes is None:
            continue
        ox, oy = offsets[i]
        for b in res.boxes:
            ax1, ay1, ax2, ay2 = b.xyxy[0]
            auto_box = (int(ox+ax1), int(oy+ay1), int(ox+ax2), int(oy+ay2))
            for v in vehicle_boxes:
                if iou(auto_box, v[:4]) >= IOU_MATCH and float(b.conf) > v[5] + CONF_MARGIN:
                    v[4] = "auto"
                    v[5] = float(b.conf)

    # -------- SIGNAL INFERENCE (FIXED LOGIC) --------
    behind = []
    for _, _, _, y2, _, _, tid in vehicle_boxes:
        if y2 < STOP_LINE_Y:
            behind.append((tid, y2))

    behind.sort(key=lambda x: STOP_LINE_Y - x[1])
    front = behind[:FRONT_N]

    stopped = 0
    for tid, fy in front:
        if tid in last_front_y:
            if abs(fy - last_front_y[tid]) < STOPPED_DIST_PX:
                stopped += 1

    if green_hold > 0:
        signal_state = "GREEN"
        green_hold -= 1
    else:
        if len(front) > 0 and stopped / len(front) >= RED_RATIO_THRESHOLD:
            signal_state = "RED"
        else:
            signal_state = "GREEN"
            green_hold = GREEN_HOLD_FRAMES

    # -------- QUEUE LENGTH --------
    queue_ids = set()
    for _, _, _, y2, _, _, tid in vehicle_boxes:
        if tid in last_front_y:
            stopped = abs(y2 - last_front_y[tid]) < STOPPED_DIST_PX
            behind_stop = y2 < STOP_LINE_Y
            in_zone = y2 > STOP_LINE_Y - QUEUE_DEPTH_PX
            if stopped and behind_stop and in_zone:
                queue_ids.add(tid)

    queue_length = len(queue_ids)

    # -------- VIOLATION CHECK + DRAW --------
    for x1, y1, x2, y2, label, conf, tid in vehicle_boxes:
        speed_px = 0
        if tid in last_front_y:
            speed_px = abs(y2 - last_front_y[tid])

        if speed_px < STOPPED_DIST_PX:
            was_stopped[tid] = True

        crossed = (
            tid in last_front_y and
            last_front_y[tid] < STOP_LINE_Y and
            y2 >= STOP_LINE_Y
        )

        if (
            crossed and
            signal_state == "RED" and
            tid not in violators and
            was_stopped.get(tid, False)
        ):
            violators.add(tid)
            violation_count += 1

            violation_log.append({
                "violation_id": violation_count,
                "vehicle_id": tid,
                "vehicle_type": label,
                "time": time_sec,
                "speed_px": round(speed_px, 2),
                "queue_length": queue_length,
                "signal_state": signal_state
            })

        last_front_y[tid] = y2

        color = (0,0,255) if tid in violators else (0,255,0)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(
            frame,
            f"{label.upper()} ID:{tid}",
            (x1, max(25, y1-5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2
        )

    # -------- OVERLAYS --------
    cv2.line(frame, (0, STOP_LINE_Y), (W, STOP_LINE_Y), (0,0,255), 3)
    cv2.putText(frame, f"SIGNAL: {signal_state}", (30,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.1,
                (0,255,255) if signal_state=="GREEN" else (0,0,255), 3)

    cv2.putText(frame, f"QUEUE: {queue_length}", (30,80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,0), 3)

    cv2.putText(frame, f"VIOLATIONS: {violation_count}", (30,120),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)

    out.write(frame)

# =========================================
# WRITE CSV
# =========================================
with open(CSV_PATH, "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=[
            "violation_id",
            "vehicle_id",
            "vehicle_type",
            "time",
            "speed_px",
            "queue_length",
            "signal_state"
        ]
    )
    writer.writeheader()
    writer.writerows(violation_log)

cap.release()
out.release()

print("✅ FIXED LOGIC PIPELINE COMPLETE")
print(f"🎥 Video: {OUTPUT_VIDEO}")
print(f"📄 CSV: {CSV_PATH}")


✅ FIXED LOGIC PIPELINE COMPLETE
🎥 Video: /kaggle/working/final_output_fixed_v1.mp4
📄 CSV: /kaggle/working/violations_v1.csv
